# trying different zero shot methods and different classification methods afterwards

2. Improving sentiment analysis 
  - add various zero shot methods + various after model (logistic regression, SVM etc) 
  - create a service with the above for easier integration with existing solution 
  - develop model performance visualisation 
    - design in figma
    - implement the design 
  - create a model download page 
     - create in figma 
     - implement the design 

In [ ]:
import pandas as pd
from transformers import pipeline

def sentiment_analysis(df, text_column, model_name):
    classifier = pipeline("zero-shot-classification", model=model_name)
    labels = ["positive", "neutral", "negative"]

    def classify_text(text):
        if pd.isna(text):
            return {"labels": [None], "scores": [None]}
        return classifier(text, labels)

    results = df[text_column].apply(classify_text)
    df['predicted_label'] = results.apply(lambda x: x['labels'][0])
    df['scores'] = results.apply(lambda x: x['scores'][0])

    return df

df = pd.DataFrame({
    'Review Text': ["I love the new design of your website!", "The service was okay, nothing special.", "I am not happy with the product quality.", None]
})

# DeBERTa
df_deberta = sentiment_analysis(df, 'Review Text', 'microsoft/deberta-large-mnli')
print("Results using DeBERTa:")
print(df_deberta)

# BART
df_bart = sentiment_analysis(df, 'Review Text', 'facebook/bart-large-mnli')
print("\nResults using BART:")
print(df_bart)

# Ernie
df_xlnet = sentiment_analysis(df, 'Review Text', 'MoritzLaurer/ernie-m-large-mnli-xnli')
print("\nResults using XLNet:")
print(df_xlnet)